In [1]:
require 'hdf5'
require 'nn'

In [455]:
-- Loading data
myFile = hdf5.open('../data/MM_data.hdf5','r')
data = myFile:all()
input_matrix_train = data['input_matrix_train']
input_matrix_train_embed = data['input_matrix_train_embed']
input_matrix_dev = data['input_matrix_dev']
input_matrix_dev_embed = data['input_matrix_dev_embed']
input_matrix_test = data['input_matrix_test']
myFile:close()

In [126]:
myFile = hdf5.open('../data/embeddings.hdf5','r')
data2 = myFile:all()
embeddings = data2['embeddings']
myFile:close()

In [313]:
train_output = input_matrix_train_embed:narrow(2,60,1)
nwords = train_output:size(1)
train_output = train_output:narrow(1,2,nwords-1)

In [318]:
train_input = torch.Tensor(nwords-1,10)
train_input:narrow(2,1,1):copy(input_matrix_train:narrow(2,1,1):narrow(1,2,nwords-1))
train_input:narrow(2,2,9):copy(input_matrix_train:narrow(2,2,9):narrow(1,1,nwords-1))

In [563]:
input_matrix_train:narrow(1,1,5)

 400001       0       0       0       0       0       0       0       1       0
    645       0       0       0       1       0       0       0       0       0
   7579       1       0       0       0       0       0       0       0       0
    515       0       0       0       0       1       0       0       0       0
    581       1       0       0       0       0       0       0       0       0
[torch.LongTensor of size 5x10]



In [320]:
train_output:narrow(1,1,5)

 4
 1
 5
 1
 1
[torch.DoubleTensor of size 5x1]



### Linear 1:

In [156]:
linear = nn.Sequential()
linear:add(nn.Linear(59,9))
linear:add(nn.SoftMax())

In [157]:
criterion = nn.ClassNLLCriterion()

In [159]:
train_model(train_input, train_output, linear, criterion, 59, 9, 0.01, 20, 16);

Epoch 1: 0.60036301612854	
Average Loss: -0.66707073234158	


Epoch 2: 0.59119391441345	
Average Loss: -0.71237675728118	


Epoch 3: 0.59684014320374	
Average Loss: -0.72320236849443	


Epoch 4: 0.590656042099	


Average Loss: -0.73588772172317	


...

Epoch 16: 0.70232391357422	
Average Loss: -0.76065918617728	


Epoch 17: 0.64208316802979	
Average Loss: -0.76106153387546	


Epoch 18: 0.73686122894287	
Average Loss: -0.76142982197363	


Epoch 19: 0.56427597999573	
Average Loss: -0.7617710446001	


Epoch 20: 0.59289908409119	
Average Loss: -0.76209164453552	


### Linear 2:

In [376]:
function train_model(train_input, train_output, model, criterion, din, nclass, eta, nEpochs, batchSize)
    -- Train the model with a mini batch SGD
    -- standard parameters are
    -- nEpochs = 1
    -- batchSize = 32
    -- eta = 0.01
    loss = torch.Tensor(nEpochs)

    -- To store the loss
    av_L = 0

    -- Memory allocation
    inputs_batch = torch.DoubleTensor(batchSize, din)
    targets_batch = torch.DoubleTensor(batchSize)
    outputs = torch.DoubleTensor(batchSize, nclass)
    df_do = torch.DoubleTensor(batchSize, nclass)

    for i = 1, nEpochs do
        -- timing the epoch
        timer = torch.Timer()
        av_L = 0
        
        -- mini batch loop
        for t = 1, train_input:size(1), batchSize do
            -- Mini batch data
            current_batch_size = math.min(batchSize,train_input:size(1)-t)
--             print('here1')
            
            inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
--             print('here2')
            
            targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
--             print('here3')
            
            -- reset gradients
            model:zeroGradParameters()
            --gradParameters:zero()

            -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
            outputs:narrow(1,1,current_batch_size):copy(model:forward({inputs_batch:narrow(1,1,current_batch_size):narrow(2,1,1),
                                                                        inputs_batch:narrow(1,1,current_batch_size):narrow(2,2,9)}))
--             print('here4')
            -- Average loss computation
            f = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
            
--             print('here5')
            av_L = av_L +f

            -- Backward pass
            df_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size)))
--             print('here6')
            model:backward({inputs_batch:narrow(1,1,current_batch_size):narrow(2,1,1), inputs_batch:narrow(1,1,current_batch_size):narrow(2,2,9)}, 
                            df_do:narrow(1,1,current_batch_size))
--             print('here7')
            model:updateParameters(eta)
            
        end
            
        print('Epoch '..i..': '..timer:time().real)
        loss[i] = av_L/math.floor(train_input:size(1)/batchSize)
        print('Average Loss: '.. loss[i])
       
    end
    
    return loss
end

In [326]:
LT = nn.LookupTable(400002,embed_dim)
LT.weight:narrow(1, 1, 400000):copy(embeddings)

In [369]:
ultimate_t = nn.Sequential()
t1 = nn.ParallelTable()

t1_1 = nn.Sequential()
t1_1:add(LT)
t1_1:add(nn.View(-1,50))

t1_2 = nn.Identity()

t1:add(t1_1)
t1:add(t1_2)

ultimate_t:add(t1)
ultimate_t:add(nn.JoinTable(2))

ultimate_t:add(nn.Linear(59,9))
ultimate_t:add(nn.LogSoftMax())

In [1]:
-- Sanity check:
pred = ultimate_t:forward{train_input:narrow(1,1,1):narrow(2,1,1),train_input:narrow(1,1,1):narrow(2,2,9)}

In [3]:
L = criterion:forward(pred,torch.Tensor({4}))

In [5]:
criterion:backward(pred,torch.Tensor({4}))

 0  0  0 -1  0  0  0  0  0
[torch.DoubleTensor of size 1x9]



In [416]:
train_input:narrow(1,1,1):narrow(2,1,1)

 645
[torch.DoubleTensor of size 1x1]



In [377]:
loss_1 = train_model(train_input, train_output, ultimate_t, criterion, 10, 9, 0.01, 20, 32)

Epoch 1: 78.489984989166	
Average Loss: 0.52836980165634	


Epoch 2: 79.068855047226	
Average Loss: 0.36818426410206	


Epoch 3: 77.054080963135	
Average Loss: 0.33124513835614	


Epoch 4: 77.238839149475	
Average Loss: 0.31238815111166	


Epoch 5: 72.342820882797	
Average Loss: 0.30003632712975	


...

Epoch 16: 72.322520971298	
Average Loss: 0.24943696050847	


Epoch 17: 70.433362960815	
Average Loss: 0.2471189986981	


Epoch 18: 70.505609989166	
Average Loss: 0.24493939147794	


Epoch 19: 70.802218914032	
Average Loss: 0.24287882398742	


Epoch 20: 70.813380002975	
Average Loss: 0.24092130150087	


In [493]:
loss_2 = train_model(train_input, train_output, ultimate_t, criterion, 10, 9, 0.01, 5, 32)

Epoch 1: 72.436408996582	
Average Loss: 0.23905349760884	


Epoch 2: 70.821997880936	
Average Loss: 0.23726421572767	


Epoch 3: 71.64350605011	
Average Loss: 0.23554399788073	


Epoch 4: 70.697922945023	
Average Loss: 0.23388478192972	


Epoch 5: 70.485579967499	
Average Loss: 0.23227964487719	


### Viterbi:

In [530]:
function compute_logscore(observations, i, model, C)
    local y = torch.zeros(C,C)
    local hot_1 = torch.zeros(C)
    for j = 1, C do
        hot_1:zero()
        hot_1[j] = 1
        y:narrow(1,j,1):copy(model:forward({observations[i]:view(1,1),hot_1:view(1,9)}))
    end
    return y
end

function viterbi(observations, compute_logscore, model, C)
    
    local y = torch.zeros(C,C)
    -- Formating tensors
    local initial = torch.zeros(C, 1)
    -- initial started with a start of sentence: <t>

    initial[{8,1}] = 1
    initial:log()

    -- number of classes
    local n = observations:size(1)
    local max_table = torch.Tensor(n, C)
    local backpointer_table = torch.Tensor(n, C)
    -- first timestep
    -- the initial most likely paths are the initial state distribution
    -- NOTE: another unnecessary Tensor allocation here
    local maxes, backpointers = (initial + compute_logscore(observations, 1, model, C)[8]):max(2)
    max_table[1] = maxes
    -- remaining timesteps ("forwarding" the maxes)
    for i=2,n do
        -- precompute edge scores
       
        y:copy(compute_logscore(observations, i, model, C))
        scores = y:transpose(1,2) + maxes:view(1, C):expand(C, C)

        -- compute new maxes (NOTE: another unnecessary Tensor allocation here)
        maxes, backpointers = scores:max(2)

        -- record
        max_table[i] = maxes
        backpointer_table[i] = backpointers
    end
    -- follow backpointers to recover max path
    local classes = torch.Tensor(n)
    maxes, classes[n] = maxes:max(1)
    for i=n,2,-1 do
        classes[i-1] = backpointer_table[{i, classes[i]}]
    end

    return classes
end

In [449]:
function compute_score(predicted_classes, true_classes)
    local n = predicted_classes:size(1)
    local right_pred = 0
    local positive_true = 0
    local positive_pred = 0
    for i=1,n do
        if predicted_classes[i] > 1 then
            positive_pred = positive_pred + 1
        end
        if true_classes[i] > 1 then
            positive_true = positive_true + 1
        end
        if (true_classes[i] == predicted_classes[i]) and true_classes[i] > 1 then
            right_pred = right_pred + 1
        end
    end
    print(positive_true)
    print(positive_pred)
    print(right_pred)
    local precision = right_pred/positive_pred
    local recall = right_pred/positive_true
    return precision, recall
end
        
function f_score(predicted_classes, true_classes)
    local p,r = compute_score(predicted_classes, true_classes)
    return 2*p*r/(p+r)
end

In [545]:
observations = input_matrix_dev:narrow(2,1,1):clone()

In [544]:
true_classes = input_matrix_dev_embed:narrow(2,60,1):squeeze()

In [546]:
cl = viterbi(observations, compute_logscore, ultimate_t, 9)

here	


In [548]:
f_tensor = f_score(cl, true_classes)
print(f_tensor)

7016	
6528	
5497	
0.81172474896633	


### Predicting on test:

In [559]:
observations_test = input_matrix_test:view(input_matrix_test:size(1),1)

In [561]:
v_seq_test = viterbi(observations_test, compute_logscore, ultimate_t, 9)

here	


In [562]:
-- Saving predicted sequence on test
myFile = hdf5.open('../submission/v_seq_test_mem', 'w')
myFile:write('v_seq_test', v_seq_test)
myFile:close()

## CAP:

In [564]:
-- Loading data
myFile = hdf5.open('../data/MM_data_cap.hdf5','r')
data = myFile:all()
input_matrix_train_cap = data['input_matrix_train_cap']
input_matrix_dev_cap = data['input_matrix_dev_cap']
input_matrix_test_cap = data['input_matrix_test_cap']
myFile:close()

In [566]:
nwords = train_output:size(1)
train_input_cap = torch.Tensor(nwords-1,15)
train_input_cap:narrow(2,1,1):copy(input_matrix_train_cap:narrow(2,1,1):narrow(1,2,nwords-1))
train_input_cap:narrow(2,2,9):copy(input_matrix_train_cap:narrow(2,2,9):narrow(1,1,nwords-1))
train_input_cap:narrow(2,11,5):copy(input_matrix_train_cap:narrow(2,11,5):narrow(1,1,nwords-1))

In [568]:
ultimate_t_cap = nn.Sequential()
t1_cap = nn.ParallelTable()

t1_cap_1 = nn.Sequential()
t1_cap_1:add(LT)
t1_cap_1:add(nn.View(-1,50))

t1_cap_2 = nn.Identity()

t1_cap:add(t1_cap_1)
t1_cap:add(t1_cap_2)

ultimate_t_cap:add(t1_cap)
ultimate_t_cap:add(nn.JoinTable(2))

ultimate_t_cap:add(nn.Linear(64,9))
ultimate_t_cap:add(nn.LogSoftMax())

In [571]:
ultimate_t_cap:forward{train_input_cap:narrow(1,1,1):narrow(2,1,1),train_input_cap:narrow(1,1,1):narrow(2,2,14)}

-2.3609 -2.3948 -1.7501 -2.2660 -2.1834 -2.4742 -1.9729 -1.9689 -2.7898
[torch.DoubleTensor of size 1x9]



In [573]:
function train_model_cap(train_input, train_output, model, criterion, din, nclass, eta, nEpochs, batchSize)
    -- Train the model with a mini batch SGD
    -- standard parameters are
    -- nEpochs = 1
    -- batchSize = 32
    -- eta = 0.01
    loss = torch.Tensor(nEpochs)

    -- To store the loss
    av_L = 0

    -- Memory allocation
    inputs_batch = torch.DoubleTensor(batchSize, din)
    targets_batch = torch.DoubleTensor(batchSize)
    outputs = torch.DoubleTensor(batchSize, nclass)
    df_do = torch.DoubleTensor(batchSize, nclass)

    for i = 1, nEpochs do
        -- timing the epoch
        timer = torch.Timer()
        av_L = 0
        
        -- mini batch loop
        for t = 1, train_input:size(1), batchSize do
            -- Mini batch data
            current_batch_size = math.min(batchSize,train_input:size(1)-t)
--             print('here1')
            
            inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
--             print('here2')
            
            targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
--             print('here3')
            
            -- reset gradients
            model:zeroGradParameters()
            --gradParameters:zero()

            -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
            outputs:narrow(1,1,current_batch_size):copy(model:forward({inputs_batch:narrow(1,1,current_batch_size):narrow(2,1,1),
                                                                        inputs_batch:narrow(1,1,current_batch_size):narrow(2,2,14)}))
--             print('here4')
            -- Average loss computation
            f = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
            
--             print('here5')
            av_L = av_L +f

            -- Backward pass
            df_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size)))
--             print('here6')
            model:backward({inputs_batch:narrow(1,1,current_batch_size):narrow(2,1,1), inputs_batch:narrow(1,1,current_batch_size):narrow(2,2,14)}, 
                            df_do:narrow(1,1,current_batch_size))
--             print('here7')
            model:updateParameters(eta)
            
        end
            
        print('Epoch '..i..': '..timer:time().real)
        loss[i] = av_L/math.floor(train_input:size(1)/batchSize)
        print('Average Loss: '.. loss[i])
       
    end
    
    return loss
end

In [574]:
loss_cap = train_model_cap(train_input_cap, train_output, ultimate_t_cap, criterion, 15, 9, 0.01, 20, 32)

Epoch 1: 70.712356090546	
Average Loss: 0.50310520465441	


Epoch 2: 70.425627946854	
Average Loss: 0.34053567456718	


Epoch 3: 70.346951007843	
Average Loss: 0.30468165034808	


Epoch 4: 71.26979804039	
Average Loss: 0.28644514853156	


Epoch 5: 70.6217648983	
Average Loss: 0.27451383334762	


...

Epoch 16: 70.745599031448	
Average Loss: 0.22754180283866	


Epoch 17: 70.9817070961	
Average Loss: 0.22547400392261	


Epoch 18: 71.781083106995	
Average Loss: 0.22353160058383	


Epoch 19: 70.286765098572	
Average Loss: 0.22169641716466	


Epoch 20: 70.86462688446	
Average Loss: 0.21995358255235	


In [606]:
loss_cap_2 = train_model_cap(train_input_cap, train_output, ultimate_t_cap, criterion, 15, 9, 0.001, 3, 32)

Epoch 1: 71.504629850388	
Average Loss: 0.21981077053702	


Epoch 2: 79.878816127777	
Average Loss: 0.21949923248368	


Epoch 3: 73.816610097885	
Average Loss: 0.21927489435913	


In [607]:
loss_cap_3 = train_model_cap(train_input_cap, train_output, ultimate_t_cap, criterion, 15, 9, 0.1, 2, 32)

Epoch 1: 72.5975689888	
Average Loss: 0.20909823798932	


Epoch 2: 70.560930013657	
Average Loss: 0.19788823075198	


In [610]:
loss_cap_4 = train_model_cap(train_input_cap, train_output, ultimate_t_cap, criterion, 15, 9, 0.1, 5, 32)

Epoch 1: 71.19276714325	
Average Loss: 0.18782437035191	


Epoch 2: 70.893792867661	
Average Loss: 0.17851937297793	


Epoch 3: 70.709676980972	
Average Loss: 0.16980573348033	


Epoch 4: 72.024089097977	
Average Loss: 0.1616779017535	


Epoch 5: 70.403036117554	
Average Loss: 0.154168990045	


In [613]:
loss_cap_5 = train_model_cap(train_input_cap, train_output, ultimate_t_cap, criterion, 15, 9, 0.1, 5, 32)

Epoch 1: 72.541322946548	
Average Loss: 0.14732098656081	


Epoch 2: 70.828729867935	
Average Loss: 0.14119754426576	


Epoch 3: 72.119047164917	
Average Loss: 0.1358135003866	


Epoch 4: 70.52068901062	
Average Loss: 0.13108787825574	


Epoch 5: 72.093950033188	
Average Loss: 0.12692133195558	


In [585]:
function compute_logscore_cap(observations, caps, i, model, C)
    local y = torch.zeros(C,C)
    local hot_1 = torch.zeros(C+5)
    for j = 1, C do
        hot_1:zero()
        hot_1[j] = 1
        hot_1:narrow(1,10,5):copy(caps:narrow(1,i,1))
        y:narrow(1,j,1):copy(model:forward({observations[i]:view(1,1),hot_1:view(1,C+5)}))
    end
    return y
end

function viterbi_cap(observations, caps, compute_logscore, model, C)
    
    local y = torch.zeros(C,C)
    -- Formating tensors
    local initial = torch.zeros(C, 1)
    -- initial started with a start of sentence: <t>

    initial[{8,1}] = 1
    initial:log()

    -- number of classes
    local n = observations:size(1)
    local max_table = torch.Tensor(n, C)
    local backpointer_table = torch.Tensor(n, C)
    -- first timestep
    -- the initial most likely paths are the initial state distribution
    -- NOTE: another unnecessary Tensor allocation here
    local maxes, backpointers = (initial + compute_logscore_cap(observations, caps, 1, model, C)[8]):max(2)
    max_table[1] = maxes
    -- remaining timesteps ("forwarding" the maxes)
    for i=2,n do
        -- precompute edge scores
       
        y:copy(compute_logscore_cap(observations, caps, i, model, C))
        scores = y:transpose(1,2) + maxes:view(1, C):expand(C, C)

        -- compute new maxes (NOTE: another unnecessary Tensor allocation here)
        maxes, backpointers = scores:max(2)

        -- record
        max_table[i] = maxes
        backpointer_table[i] = backpointers
    end
    -- follow backpointers to recover max path
    local classes = torch.Tensor(n)
    maxes, classes[n] = maxes:max(1)
    for i=n,2,-1 do
        classes[i-1] = backpointer_table[{i, classes[i]}]
    end

    return classes
end

In [602]:
observations_dev = input_matrix_dev_cap:narrow(2,1,1):clone()
dev_cap = input_matrix_dev_cap:narrow(2,11,5)
dev_true_classes = input_matrix_dev_cap:narrow(2,16,1):squeeze()

In [611]:
cl_caps = viterbi_cap(observations_dev, dev_cap, compute_logscore_cap, ultimate_t_cap, 9)

In [612]:
f_tensor_caps = f_score(cl_caps, dev_true_classes)
print(f_tensor_caps)

7016	
6594	
5630	
0.82733284349743	


In [614]:
cl_caps_ = viterbi_cap(observations_dev, dev_cap, compute_logscore_cap, ultimate_t_cap, 9)

In [615]:
f_tensor_caps_ = f_score(cl_caps, dev_true_classes)
print(f_tensor_caps)

7016	
6594	
5630	
0.82733284349743	


## Predicting:

In [620]:
observations_test_cap = input_matrix_test_cap:narrow(2,1,1)
observations_test_caps = input_matrix_test_cap:narrow(2,2,5)

In [621]:
v_seq_test_cap = viterbi_cap(observations_test_cap, observations_test_caps, compute_logscore_cap, ultimate_t_cap, 9)

In [622]:
-- Saving predicted sequence on test
myFile = hdf5.open('../submission/v_seq_test_mem_cap', 'w')
myFile:write('v_seq_test', v_seq_test_cap)
myFile:close()

In [634]:
viterbi(observations:narrow(1,1,5), compute_logscore, ultimate_t, 9)

here	


 8
 1
 1
 1
 1
[torch.DoubleTensor of size 5]



In [635]:
true_classes:narrow(1,1,5)

 8
 1
 1
 4
 1
[torch.DoubleTensor of size 5]

